In [ ]:
from datetime import datetime

from pytket import Circuit

import qnexus as qnx


from nexus_dataclasses.backend_config import QuantinuumConfig

In [ ]:
my_project = qnx.projects.get(name="VanyaTest")

qnx.context.set_active_project(my_project)
    
active_jobs = qnx.jobs.filter(
    project_ref=my_project,
    status=["SUBMITTED", "QUEUED", "RUNNING"]
)

active_jobs.all().summarize()

In [ ]:
c = Circuit(2,2).H(0).CX(0,1).measure_all()


my_project_name = str(datetime.now())
my_circuit_name = "circuit_1"
my_job_name_prefix = str(datetime.now())

# TODO this is sort of what users do in their scripts when it comes to resuming

try:
    project_ref = qnx.projects.get(name=my_project_name)
except Exception:
    project_ref = qnx.projects.create(
        name=my_project_name
    )



with qnx.context.using_project(project_ref):

    # Prepare the circuit in Nexus
    try:
        my_circuit_ref = qnx.circuits.get(name=my_circuit_name)
    except Exception:
        my_circuit_ref = qnx.circuits.create(
            circuit=c,
            name=my_circuit_name,
        )
    
    # Resume the compilation in Nexus
    try:
        compile_job_ref = qnx.jobs.get(
            name=my_job_name_prefix,
            type=qnx.jobs.JobType.Compile
        )
    except Exception:
        compile_job_ref = qnx.jobs.compile.run(
            circuits=[my_circuit_ref],
            name=f"{my_job_name_prefix}_compile",
            optimisation_level=1,
            target=QuantinuumConfig(device_name="H1-1LE")
        )
    if compile_job_ref.last_status in qnx.jobs.WAITING_STATUS:
         qnx.jobs.wait_for(compile_job_ref).status
    elif compile_job_ref.last_status == qnx.jobs.StatusEnum.COMPLETED:
        pass
    else:
        raise Exception(f"Waiting on compile job: {compile_job_ref}")
    compiled_circuits = qnx.jobs.compile.results(compile_job_ref)

    # Resume the execution
    try:
        execute_job_ref = qnx.jobs.get(
            name=f"{my_job_name_prefix}_execute",
            type=qnx.jobs.JobType.Execute
        )
    except Exception:
        execute_job_ref = qnx.jobs.execute.run(
            circuits=compiled_circuits,
            name=f"{my_job_name_prefix}_execute",
            n_shots=[100],
            target=QuantinuumConfig(device_name="H1-1LE")
        )
    if execute_job_ref.last_status in qnx.jobs.WAITING_STATUS:
        qnx.jobs.wait_for(execute_job_ref).status
    elif execute_job_ref.last_status == qnx.jobs.StatusEnum.COMPLETED:
        pass
    else:
        raise Exception(f"Waiting on execute job: {execute_job_ref}")
    
    results = qnx.jobs.execute.results(execute_job_ref)
    
    
